Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

In [1]:
import retinoto_py as fovea
# args = fovea.Params(do_fovea=True, n_train_stop=2_000, n_val_stop=1_000)
args = fovea.Params(do_fovea=True, n_train_stop=200_000, n_val_stop=10_000)
# args = fovea.Params(do_fovea=True, n_train_stop=0, n_val_stop=0, in_memory=False)
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	 user laurent	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 1998 has been set.


Params(image_size=224, do_mask=False, do_fovea=True, rs_min=0.0, rs_max=-5.0, padding_mode='zeros', seed=1998, batch_size=80, num_workers=4, in_memory=True, model_name='resnet101', do_scratch=False, num_epochs=10, n_train_stop=200000, n_val_stop=10000, lr=0.001, delta1=0.05, delta2=0.0, weight_decay=0.003, label_smoothing=0.0001, shuffle=True, verbose=True)

In [2]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings(
    "ignore",
    message=r"iCCP: profile 'ICC Profile'",
    category=UserWarning,
    module='PIL.PngImagePlugin'
)

# testing each network on the validation dataset

In [3]:
%ls cached_data/32*

cached_data/32_fovea_model_name=resnet101_dataset=bbox_retrained_bkp.json
cached_data/32_fovea_model_name=resnet101_dataset=bbox_retrained_bkp.pth
cached_data/32_fovea_model_name=resnet101_dataset=bbox_retrained.lock
cached_data/32_fovea_model_name=resnet101_dataset=full_retrained.json
cached_data/32_fovea_model_name=resnet101_dataset=full_retrained.pth


In [ ]:
# %rm "cached_data/32_fovea_model_name=resnet101_dataset=bbox"  # FORCING RECOMPUTE
# %rm "cached_data/32_fovea_model_name=resnet101_dataset=bbox_retrained."*  # FORCING RECOMPUTE
# %rm cached_data/32_fovea_model_*  # FORCING RECOMPUTE
# %rm cached_data/32_fovea_model_*.lock  # FORCING RECOMPUTE
model_name = args.model_name
for dataset in fovea.all_datasets:
    name = f'32_fovea_model_name={model_name}_dataset={dataset}_retrained'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)
    print(50*'.')

Load JSON from pre-trained resnet cached_data/32_fovea_model_name=resnet101_dataset=full_retrained.json
cached_data/32_fovea_model_name=resnet101_dataset=full_retrained.pth: accuracy = 0.703
..................................................


Putting images in memory:   0%|          | 0/200000 [00:00<?, ?it/s]

libpng warning: iCCP: profile 'ICC Profile': 'desc': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'wtpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bkpt': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'rXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'gXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'bXYZ': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmnd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'dmdd': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'vued': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'view': ICC profile tag start not a multiple of 4
libpng warning: iCCP: profile 'ICC Profile': 'lumi

Putting images in memory:   0%|          | 0/10000 [00:00<?, ?it/s]

Training model resnet101, file= cached_data/32_fovea_model_name=resnet101_dataset=bbox_retrained.pth - image_size=224
Model has 1000 output features to final FC layer for 1000 classes.
Starting learning...


...

epoch=6/10:   0%|          | 0/2500 [00:00<?, ?it/s]

Vat @Epoch 6/10:   0%|          | 0/125 [00:00<?, ?it/s]

epoch=7/10:   0%|          | 0/2500 [00:00<?, ?it/s]

Vat @Epoch 7/10:   0%|          | 0/125 [00:00<?, ?it/s]

epoch=8/10:   0%|          | 0/2500 [00:00<?, ?it/s]

## Model comparison

In [ ]:
model_name = args.model_name
for dataset in fovea.all_datasets:
    results = fovea.pd.read_json(args.data_cache / f'32_fovea_model_name={model_name}_dataset={dataset}_retrained.json')
    print(model_name, dataset, results.tail(1)['acc_val'])

In [ ]:

len(results)

In [ ]:
results.tail(1)

In [ ]:
results.tail(1)['acc_val']

In [ ]:
results_13 = fovea.pd.read_json(args.data_cache / '13_model_comparison.json')
results_13

In [ ]:
%ls cached_data/20*

In [ ]:
model_name, dataset

In [ ]:
# results_20 = fovea.pd.read_json(args.data_cache / '20_model_name={model_name}_dataset={dataset}_retrained.json')
# results_20

In [ ]:
# for dataset in fovea.all_datasets:
#     name = f'20_model_name={model_name}_dataset={dataset}_retrained'
#     model_filename, json_filename = fovea.do_learning(args, dataset, name)
#     results = fovea.pd.read_json(json_filename)
#     print(results)

## Plot learning evolution

In [ ]:
fig, ax = fovea.plt.subplots()
for dataset in ['bbox']:#fovea.all_datasets:
    color = 'r'
    lw = 1

    # name = f'32_fovea_model_name={model_name}_dataset={dataset}_retrained'
    # cached_data/32_fovea_model_name=resnet101_dataset=bbox_retrained.json
    json_filename = args.data_cache / f'32_fovea_model_name={model_name}_dataset={dataset}_retrained.json'
    # model_filename, json_filename = fovea.do_learning(args, dataset, name)

    df_train = fovea.pd.read_json(json_filename, orient='records')

    # df_train_roll = df_train.rolling(window=5, min_periods=1, center=False).mean()
    ax = df_train.plot(x='total_image', y='acc_train', 
                        c=color, ls='dashed', lw=lw,
                        grid=True, ax=ax, label='TRAIN: ' + args.model_name)    
    ax = df_train.plot(x='total_image', y='acc_val', 
                        c=color, lw=lw,
                        grid=True, ax=ax, label='VAL: ' + args.model_name)   

In [ ]:
df_train

In [ ]:
json_filename